In [4]:
import numpy as np
from scipy.optimize import linear_sum_assignment

cities = ['A', 'B', 'C', 'D', 'E']
n = len(cities)
city_index = {c: i for i, c in enumerate(cities)}

# Matriz de distancias
dist = np.full((n, n), np.inf)
np.fill_diagonal(dist, 0)

edges = [
    ('A', 'B', 5), ('A', 'C', 5), ('A', 'D', 6), ('A', 'E', 4),
    ('B', 'C', 3), ('B', 'D', 7), ('B', 'E', 6),
    ('C', 'D', 4), ('C', 'E', 8),
    ('D', 'E', 5)
]

for u, v, w in edges:
    i, j = city_index[u], city_index[v]
    dist[i][j] = w
    dist[j][i] = w

# Parámetros ajustados
PRF = 2000
PRC = 2000
PD = 50
dt = 0.01
max_iter = 10000

# Inicialización con ruido controlado
V = np.random.rand(n, n) * 0.1 + 0.1

# Simulación
for _ in range(max_iter):
    for i in range(n):
        for j in range(n):
            sum_row = np.sum(V[i, :]) - V[i, j]
            sum_col = np.sum(V[:, j]) - V[i, j]

            next_j = (j + 1) % n
            prev_j = (j - 1) % n
            sum_dist = 0
            for k in range(n):
                sum_dist += dist[i, k] * (V[k, next_j] + V[k, prev_j])

            U_ij = -PRF * sum_row - PRC * sum_col - PD * sum_dist
            V[i, j] += dt * U_ij
            V[i, j] = 1 / (1 + np.exp(-V[i, j]))  # Sigmoide

# Binarización estricta
V = (V > 0.8).astype(float)

# Extraer ruta con algoritmo húngaro
row_ind, col_ind = linear_sum_assignment(-V)
tour = [cities[i] for i in row_ind]

# Verificar validez
if len(set(tour)) == n:
    print("Tour válido:", tour)
else:
    print("Ajustar PRF/PRC o iteraciones.")

Tour válido: ['A', 'B', 'C', 'D', 'E']
